In [2]:
import time
from openai import OpenAI

In [3]:
question = f'''Gustavo está fazendo um tratamento de saúde e deve tomar 3 tipos diferentes de remédios. O primeiro remédio deve ser tomado de 4 em 4 horas, o segundo de 6 em 6 horas e o terceiro de 8 em 8 horas. Se Gustavo tomar os três remédios juntos, em quantas horas irá tomar os três juntos novamente?'''

official_answer_key = "vinte e quatro"

assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="""
  Você é um tutor pessoal de matemática. Sua tarefa é gerar um código Python para resolver a questão matemática que vou fornecer.
  Atenha-se a fornecer apenas o código Python necessário para resolver o problema, sem explicações ou comentários adicionais.
  Certifique-se de que o código seja funcional e eficiente, levando em consideração boas práticas de programação em Python.
  """,
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=question
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

python_code = mensagens.data[0].content[0].text.value

NameError: name 'client' is not defined

In [ ]:
# Run Step by Step
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

for step in run_steps.data[::-1]:
    print('======= Step >', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('```')
            print(tool_call.code_interpreter.input)
            print('```')
            if tool_call.code_interpreter.outputs:
                if tool_call.code_interpreter.outputs[0].type == 'logs':
                    print('Result')
                    print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == 'text':
            print(message.content[0].text.value)

In [ ]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=f"""Você deve executar o seguinte código Python: {python_code}. Não faça modificações ou adições ao código, e não forneça explicações ou saídas detalhadas. Apenas execute o código e retorne o resultado da execução."""
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

execution_result = mensagens.data[0].content[0].text.value
print(execution_result)


24


In [ ]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=f'''Compare o seguinte resultado: {execution_result} obtido pelo código Python: {python_code} com o gabarito oficial: {official_answer_key}. O resultado fornecido pelo código Python é suficiente para alcançar a resposta do gabarito oficial? Responda apenas com "SIM" ou "NÃO".'''
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

execution_result = mensagens.data[0].content[0].text.value
print(execution_result)


SIM


In [ ]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=f'''
  Você é um especialista em Matemática sua tarefa é fornecer soluções detalhadas e explicações claras, bem documentadas. Seu papel é guiar o aluno para uma compreensão profunda do tema abordado e da resposta correta, através de explicações detalhadas e passo a passo. Você interage com o aluno de forma educada e gentil, evitando repetir o enunciado da questão. Suas respostas são elaboradas para facilitar o entendimento integral do assunto em discussão. Com sua experiência, você fornece uma aprendizagem que verdadeiramente dialoga com as necessidades do aluno.
  Escreva um gabarito comentado sobre seguinte questão: {question} seguindo os passos abaixo:
  - Explique o tema da questão, como ela aborda o tema e os conhecimentos necessários para resolvê-la.
  - Justifique a resposta correta.
  - Comece sua mensagem deixando claro qual é a resposta correta e depois desenvolva e explique a questão.
  - Mostre o passo a passo de como resolver matemáticamente o problema.'''
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

execution_result = mensagens.data[0].content[0].text.value
print(execution_result)

### Gabarito Comentado: Frequência de Medicação

A resposta correta é: **24 horas**.

Agora, vamos explicar a questão passo a passo:

#### Tema da Questão

Esta questão aborda o conceito de múltiplos comuns em matemática, especificamente o **mínimo múltiplo comum (MMC)**. O MMC de dois ou mais números é o menor número inteiro positivo que é múltiplo de cada um dos números. Esse conceito é frequentemente usado para sincronizar eventos ou ciclos que ocorrem em diferentes períodos.

##### Conhecimentos Necessários

1. **Múltiplos e Divisores:** Entender quais são os múltiplos de um número.
2. **Fatoração de Números:** Saber fatorar números em fatores primos.
3. **Mínimo Múltiplo Comum:** Compreender como encontrar o MMC de dois ou mais números.

#### Justificativa da Resposta

Para determinar em quantas horas Gustavo tomará os três remédios simultaneamente novamente, é necessário calcular o MMC dos períodos de tempo em que ele toma cada remédio: 4 horas, 6 horas e 8 horas. Este mínimo múl